In [3]:
import os
from tensorflow import keras
from keras.utils import load_img
from keras.utils import img_to_array
import numpy as np
import cv2

In [4]:
from csv import reader

# Load a CSV file
def load_csv(filename):
	file = open(filename, "r")
	lines = reader(file)
	dataset = list(lines)
	return dataset

In [5]:
PROCESSED_FOLDER = 'processed/'

dataset  =[]
train_ds=[]
valid_ds=[]

# 將驗證碼切成四碼
# 先長後寬 先左上角後右下角
split0 = ( 20, 0, 45,48)
split1 = ( 45, 0, 70,48)
split2 = ( 70, 0, 95,48)
split3 = ( 95, 0,120,48)

for i in range(101,10101):
    path = PROCESSED_FOLDER + str(i) + '.jpg'
    img = cv2.imread(path)
    dataset.append(img[split0[1]:split0[3], split0[0]: split0[2]])
    dataset.append(img[split1[1]:split1[3], split1[0]: split1[2]])
    dataset.append(img[split2[1]:split2[3], split2[0]: split2[2]])
    dataset.append(img[split3[1]:split3[3], split3[0]: split3[2]])

np.array(dataset).shape

train_ds = np.array(dataset[5000:40000])
valid_ds = np.array(dataset[0:5000])

train_ds = np.array(train_ds)
valid_ds = np.array(valid_ds)

print(train_ds.shape,' ',valid_ds.shape)


(35000, 48, 25, 3)   (5000, 48, 25, 3)


In [6]:
# allow chars, chars to one hot , len(allow chars)
def one_hot (allow, char, length):
    init=[]
    for i in range(length):
        init.append(0)
    # print(allow.indes(char))
    init[allow.index(char)]=1
    # print(init)
    return init

In [21]:
size = ( 48, 140)
allow_chars = ['H', 'R', '3', '7', '5', 'Y', 'F', 'Z', 'C', '9', 'Q', 'T', 'N', '2', 'P', 'A', 'K', 'M', '4', '6', 'V', 'D', 'W', 'G']

# CSV loading

labelset =[]
train_lb=[]
valid_lb=[]

labels = load_csv('./label.csv')

# print(np.array(labels).shape)
# input()

for i in range(100,10100):
    for j in labels[i]:
        for k in j:
            labelset.append(k)

for i in range(len(labelset)):
    labelset[i] = one_hot(allow_chars,labelset[i],len(allow_chars))
# print(np.array(labelset).shape)

# print(len(labelset))
train_lb = np.array(labelset[5000:40000])
valid_lb = np.array(labelset[0:5000])

print(train_lb.shape,' ',valid_lb.shape)

(35000, 24)   (5000, 24)


In [23]:
from buildmodel import build_model

model = build_model(25,48,allow_chars)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history_model1=model.fit(
    train_ds,
    train_lb,
    batch_size=32,
    epochs=10,
    validation_split=0.2,
    validation_data=(valid_ds,valid_lb)
)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 48, 25, 3)]       0         
                                                                 
 conv2d_8 (Conv2D)           (None, 48, 25, 32)        896       
                                                                 
 conv2d_9 (Conv2D)           (None, 48, 25, 32)        9248      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 24, 12, 32)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 24, 12, 64)        18496     
                                                                 
 conv2d_11 (Conv2D)          (None, 24, 12, 64)        36928     
                                                           

In [ ]:
from keras.models import load_model

model.save('model.h5')  # creates a HDF5 file 'model.h5'


# 驗證是否成功

In [ ]:
# from keras.models import load_model
# from tensorflow import keras
# import numpy as np
# # 載入模型
# model = load_model('my_model.h5')

In [ ]:

# def decode (de):
#     output=[]
#     allow_chars = ['H', 'R', '3', '7', '5', 'Y', 'F', 'Z', 'C', '9', 'Q', 'T', 'N', '2', 'P', 'A', 'K', 'M', '4']
#     for i in range(4):
#         j = de[i]
#         output.append(allow_chars[j])
#     return output

In [ ]:
# de = np.argmax(model.predict(train_ds[12:16]),axis=1)
# print(decode(de))

1/1 [==============================] - 0s 31ms/step
['3', 'C', 'R', '9']
